In [30]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import pickle

In [32]:
with open('df_with_channel_region_interaction', 'rb') as f:
    df = pickle.load(f)

In [12]:
with open('target', 'rb') as f:
    target = pickle.load(f)

### Ideas ###


1. Imbalanced Data - Use SMOTE, or class weights.
2. Outlier Detection??
3. How to convert region and sales channel to categories? Based on category encoder, dummy encoding or manual binning?
4. Feature creation? What interaction effects can we include?

In [3]:
import pickle
with open('xgboost_params', 'rb') as f:
    opt_params = pickle.load(f)

In [4]:
opt_params

{'gamma': 0.9586988172910753,
 'learning_rate': 0.6751329545102683,
 'max_delta_step': 0.7129909876073443,
 'max_depth': 7,
 'n_estimators': 608,
 'reg_lambda': 0.8243424141017613,
 'subsample': 0.9007961192170519,
 'scale_pos_weight': 7.198}

In [34]:
from xgboost import XGBRFClassifier, XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [35]:
x_train, x_test, y_train, y_test = train_test_split(df, target)

In [36]:
xg = XGBRFClassifier(**opt_params)
xg.fit(x_train, y_train)

XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=1, gamma=0.9586988172910753, gpu_id=-1,
                importance_type='gain', interaction_constraints='',
                learning_rate=0.6751329545102683,
                max_delta_step=0.7129909876073443, max_depth=7,
                min_child_weight=1, missing=nan, monotone_constraints='()',
                n_estimators=608, n_jobs=0, num_parallel_tree=608,
                objective='binary:logistic', random_state=0, reg_alpha=0,
                reg_lambda=0.8243424141017613, scale_pos_weight=7.198,
                subsample=0.9007961192170519, tree_method='exact',
                validate_parameters=1, verbosity=None)

In [37]:
roc_auc_score(y_test, xg.predict(x_test))

0.4984929086032066

In [32]:
rf = RandomForestClassifier(oob_score=True, n_jobs=10, class_weight={0:8, 1:1})

In [33]:
rf.fit(x_train, y_train)
roc_auc_score(y_test, rf.predict(x_test))

0.5588495018663231

In [34]:
adb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), 100)
adb.fit(x_train, y_train)
roc_auc_score(y_test, adb.predict(x_test))

C:\Users\agast\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass n_estimators=100 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


0.5143972513793594

In [26]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedBaggingClassifier

In [27]:
balanced_bag = BalancedBaggingClassifier(DecisionTreeClassifier(max_depth=5), 300, n_jobs=10, oob_score=True)
balanced_bag.fit(x_train, y_train)

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                          n_estimators=300, n_jobs=10, oob_score=True)

In [28]:
roc_auc_score(y_test, balanced_bag.predict(x_test)) 

0.5025117597628499

In [29]:
roc_auc_score(y_train, balanced_bag.predict(x_train)) 

0.5340576237489906

In [ ]:
balanced_bag_gbm = BalancedBaggingClassifier(GradientBoostingClassifier(max_depth=5), 300, n_jobs=10, oob_score=True)
balanced_bag_gbm.fit(x_train, y_train)

In [ ]:
roc_auc_score(y_test, balanced_bag_gbm.predict(x_test)), roc_auc_score(y_train, balanced_bag_gbm.predict(x_train)) 

In [ ]:
dt = DecisionTreeClassifier(max_depth=5)
dt.fit(x_resampled, y_resampled)

In [14]:
rus = RandomUnderSampler(sampling_strategy=1)
x_train_resampled, y_train_resampled = rus.fit_resample(x_train, y_train)

In [ ]:
adb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=6), 70)
bagged_adb = BalancedBaggingClassifier(adb, n_estimators=200, n_jobs=10, oob_score=True)
bagged_adb.fit(x_train, y_train)
roc_auc_score(y_train, bagged_adb.predict(x_train)), roc_auc_score(y_test, bagged_adb.predict(x_test))

C:\Users\agast\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass n_estimators=70 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [15]:
gbm = GradientBoostingClassifier(max_depth=3, learning_rate=0.05, n_estimators=500)
gbm.fit(x_train_resampled, y_train_resampled)

GradientBoostingClassifier(learning_rate=0.05, n_estimators=500)

In [16]:
roc_auc_score(y_test, gbm.predict(x_test)) 

0.5020266286213728

In [24]:
roc_auc_score(y_train, gbm.predict(x_train))

0.5285532838499599

In [22]:
rf = RandomForestClassifier(oob_score=True, n_jobs=10)

In [23]:
rf.fit(x_train_resampled, y_train_resampled)
roc_auc_score(y_test, rf.predict(x_test))

0.49514395776096143

In [21]:
roc_auc_score(y_train, rf.predict(x_train))

0.7880489921143857

In [ ]:
xgb = XGBRFClassifier(learning_rate=0.6, subsample=0.6, n_estimators=600)

In [ ]:
xgb.fit(x_train_resampled, y_train_resampled)
roc_auc_score(y_test, xgb.predict(x_test))

In [70]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

In [ ]:
cv = cross_val_score(AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=100), x_train_resampled, \
               y_train_resampled, scoring=make_scorer(roc_auc_score), cv=5)

In [ ]:
cv

In [ ]:
adb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=9), 50)
bagged_adb = BaggingClassifier(adb, n_estimators=50, n_jobs=10)
bagged_adb.fit(x_train_resampled, y_train_resampled)
roc_auc_score(y_train_resampled, bagged_adb.predict(x_train_resampled)), roc_auc_score(y_test, bagged_adb.predict(x_test))

In [69]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Make new features 

pf = PolynomialFeatures(interaction_only=True, include_bias=False)
new_features = pf.fit_transform(df_encoded[['Gender_Male', 'Previously_Insured_1', 'Vehicle_Damage_Yes']])[:, -3:]
df_encoded[['Gender_Prev_Insured', 'Gender_Vehicle_Damage', 'Prev_Insured_Vehicle_Damage']] = new_features 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_encoded, target)

In [ ]:
rus = RandomUnderSampler(sampling_strategy=1)
x_train_resampled, y_train_resampled = rus.fit_resample(x_train, y_train)

In [ ]:
rf = RandomForestClassifier(500, oob_score=True, n_jobs=10)
rf.fit(x_train_resampled, y_train_resampled)
roc_auc_score(y_test, rf.predict(x_test))

In [ ]:
roc_auc_score(y_train, rf.predict(x_train))

In [ ]:
gbm = GradientBoostingClassifier(max_depth=10, learning_rate=0.02, n_estimators=100)
gbm.fit(x_train_resampled, y_train_resampled)

In [ ]:
roc_auc_score(y_test, gbm.predict(x_test)), roc_auc_score(y_train, gbm.predict(x_train))

In [ ]:
bagged_gbm = BaggingClassifier(GradientBoostingClassifier(max_depth=10, learning_rate=0.01, n_estimators=200), 50, n_jobs=10)
bagged_gbm.fit(x_train_resampled, y_train_resampled)

In [ ]:
roc_auc_score(y_test, bagged_gbm.predict(x_test)), roc_auc_score(y_train_resampled, bagged_gbm.predict(x_train_resampled))

In [ ]:
blagged_gbm = BalancedBaggingClassifier(GradientBoostingClassifier(max_depth=10, learning_rate=0.05, n_estimators=75), 100, n_jobs=10)
blagged_gbm.fit(x_train, y_train)

In [ ]:
roc_auc_score(y_test, blagged_gbm.predict(x_test)), roc_auc_score(y_train, blagged_gbm.predict(x_train))

In [106]:
test_df = pd.read_csv('test.csv')

In [107]:
test_ids = test_df['id']

In [108]:
test_df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,25,1,11.0,1,< 1 Year,No,35786.0,152.0,53
1,381111,Male,40,1,28.0,0,1-2 Year,Yes,33762.0,7.0,111
2,381112,Male,47,1,28.0,0,1-2 Year,Yes,40050.0,124.0,199
3,381113,Male,24,1,27.0,1,< 1 Year,Yes,37356.0,152.0,187
4,381114,Male,27,1,28.0,1,< 1 Year,No,59097.0,152.0,297
...,...,...,...,...,...,...,...,...,...,...,...
127032,508142,Female,26,1,37.0,1,< 1 Year,No,30867.0,152.0,56
127033,508143,Female,38,1,28.0,0,1-2 Year,Yes,28700.0,122.0,165
127034,508144,Male,21,1,46.0,1,< 1 Year,No,29802.0,152.0,74
127035,508145,Male,71,1,28.0,1,1-2 Year,No,62875.0,26.0,265


In [109]:
test_df.drop('id', axis=1, inplace=True)

In [ ]:
woe.transform(test_df)

In [ ]:
test_df.drop('Driving_License', axis=1, inplace=True)

In [ ]:
test_df['Vehicle_Age'] = test_df['Vehicle_Age'].map(vehicle_age_encoding)

In [ ]:
test_df['Previously_Insured'] = test_df['Previously_Insured'].astype('object')

In [ ]:
test_df = pd.get_dummies(test_df, drop_first=True)

In [ ]:
new_features = pf.transform(test_df[['Gender_Male', 'Previously_Insured_1', 'Vehicle_Damage_Yes']])[:, -3:]
test_df[['Gender_Prev_Insured', 'Gender_Vehicle_Damage', 'Prev_Insured_Vehicle_Damage']] = new_features 

In [ ]:
preds = blagged_gbm.predict(test_df)

In [ ]:
submission = pd.DataFrame([test_ids, preds]).T

In [ ]:
submission.rename(columns={ 'Unnamed 0':'Response'}, inplace=True)

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)

In [59]:
df_2

,Gender,Age,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Vintage,Response,Policy_Sales_Channel_Label,Region_Code_Labels
0,Male,44,0,> 2 Years,Yes,40454.0,217,1,4,5
1,Male,47,0,> 2 Years,Yes,38294.0,27,1,4,5
2,Female,56,0,1-2 Year,Yes,32031.0,72,1,4,5
3,Male,79,0,1-2 Year,Yes,57444.0,222,0,4,5
4,Male,39,0,1-2 Year,Yes,57582.0,141,0,4,5
...,...,...,...,...,...,...,...,...,...,...
381104,Male,42,1,1-2 Year,No,40550.0,158,0,0,1
381105,Male,45,0,1-2 Year,Yes,34972.0,224,0,0,1
381106,Male,63,1,1-2 Year,No,28128.0,148,0,0,1
381107,Male,30,1,< 1 Year,No,30339.0,37,0,0,1


In [60]:
df_2.dtypes

Gender                         object
Age                             int64
Previously_Insured             object
Vehicle_Age                    object
Vehicle_Damage                 object
Annual_Premium                float64
Vintage                         int64
Response                        int64
Policy_Sales_Channel_Label      int64
Region_Code_Labels              int32
dtype: object

In [63]:
df_encoded_2 = pd.get_dummies(df_2, drop_first=True)

In [64]:
df_encoded_2.shape

(381109, 20)

In [65]:
df_encoded_2.drop('Response', axis=1, inplace=True)

In [66]:
df_encoded_2.shape

(381109, 19)

In [67]:
df_encoded_2.rename(columns={'Vehicle_Age_< 1 Year':'Vehicle_Age_Under_1', 'Vehicle_Age_> 2 Years':'Vehicle_Age_Over_2'}, inplace=True)

In [68]:
x_train, x_test, y_train, y_test = train_test_split(df_encoded_2, target)

NameError: name 'train_test_split' is not defined

In [323]:
xgb = XGBRFClassifier(**opt_params)
xgb.fit(x_train, y_train)

XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=1, gamma=0.9586988172910753, gpu_id=-1,
                importance_type='gain', interaction_constraints='',
                learning_rate=0.6751329545102683,
                max_delta_step=0.7129909876073443, max_depth=7,
                min_child_weight=1, missing=nan, monotone_constraints='()',
                n_estimators=608, n_jobs=0, num_parallel_tree=608,
                objective='binary:logistic', random_state=0, reg_alpha=0,
                reg_lambda=0.8243424141017613, scale_pos_weight=7.198,
                subsample=0.9007961192170519, tree_method='exact',
                validate_parameters=1, verbosity=None)

In [327]:
roc_auc_score(y_test, xgb.predict(x_test)), roc_auc_score(y_train, xgb.predict(x_train))

(0.7984795765266068, 0.798043011926051)

In [338]:
balanced_bag = BalancedBaggingClassifier(DecisionTreeClassifier(max_depth=6), 50, n_jobs=10, oob_score=True)
balanced_bag.fit(x_train, y_train)

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                          n_estimators=50, n_jobs=10, oob_score=True)

In [339]:
roc_auc_score(y_test, balanced_bag.predict(x_test)), roc_auc_score(y_train, balanced_bag.predict(x_train))

(0.7974453501346718, 0.7966735715286414)

**Add new features**

In [74]:
df_encoded_2

,Age,Annual_Premium,Vintage,Gender_Male,Previously_Insured_1,Vehicle_Age_Under_1,Vehicle_Age_Over_2,Vehicle_Damage_Yes,Policy_Sales_Channel_Label_1,Policy_Sales_Channel_Label_2,Policy_Sales_Channel_Label_3,Policy_Sales_Channel_Label_4,Policy_Sales_Channel_Label_5,Policy_Sales_Channel_Label_6,Policy_Sales_Channel_Label_7,Region_Code_Labels_2,Region_Code_Labels_3,Region_Code_Labels_4,Region_Code_Labels_5
0,44,40454.0,217,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1
1,47,38294.0,27,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1
2,56,32031.0,72,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
3,79,57444.0,222,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
4,39,57582.0,141,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381104,42,40550.0,158,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
381105,45,34972.0,224,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
381106,63,28128.0,148,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
381107,30,30339.0,37,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [455]:
xgb = XGBRFClassifier(**opt_params)
xgb.fit(x_train, y_train)

XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=1, gamma=0.9586988172910753, gpu_id=-1,
                importance_type='gain', interaction_constraints='',
                learning_rate=0.6751329545102683,
                max_delta_step=0.7129909876073443, max_depth=7,
                min_child_weight=1, missing=nan, monotone_constraints='()',
                n_estimators=608, n_jobs=0, num_parallel_tree=608,
                objective='binary:logistic', random_state=0, reg_alpha=0,
                reg_lambda=0.8243424141017613, scale_pos_weight=7.198,
                subsample=0.9007961192170519, tree_method='exact',
                validate_parameters=1, verbosity=None)

In [456]:
roc_auc_score(y_test, xgb.predict(x_test)), roc_auc_score(y_train, xgb.predict(x_train))

(0.7949926907457953, 0.799596533867677)

In [ ]:
xgb.feature

In [88]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

In [89]:
rus = RandomUnderSampler()
x_undersampled, y_undersampled = rus.fit_resample(df_encoded_2, target)

In [90]:
x_train_undersampled, x_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(x_undersampled, y_undersampled)

In [94]:
y_train_undersampled.value_counts()

1    35082
0    34983
Name: Response, dtype: int64

In [95]:
y_test_undersampled.value_counts()

0    11727
1    11628
Name: Response, dtype: int64

In [96]:
logit = LogisticRegression(max_iter=1000)
logit.fit(x_train_undersampled, y_train_undersampled)

LogisticRegression(max_iter=1000)

In [97]:
roc_auc_score(y_test, logit.predict(x_test)), roc_auc_score(y_train, logit.predict(x_train))

(0.5025320781942401, 0.49928766787443335)

In [98]:
rf = RandomForestClassifier(500, n_jobs=10)
rf.fit(x_train_undersampled, y_train_undersampled)

RandomForestClassifier(n_estimators=500, n_jobs=10)

In [104]:
rf_2 = RandomForestClassifier(500, n_jobs=10)
rf_2.fit(x_undersampledersampled, y_undersampled)

RandomForestClassifier(n_estimators=500, n_jobs=10)

In [99]:
roc_auc_score(y_test, rf.predict(x_test)), roc_auc_score(y_train, rf.predict(x_train))

(0.713439147403495, 0.7125593309835498)

In [100]:
roc_auc_score(y_train_undersampled, rf.predict(x_train_undersampled))

0.9987867778686671

In [114]:
a = pd.Series(rf.feature_importances_, index=x_train.columns)

In [128]:
a[-28:]

Policy_Sales_Channel_Label_1___Region_Code_Labels_2    0.015565
Policy_Sales_Channel_Label_1___Region_Code_Labels_3    0.005172
Policy_Sales_Channel_Label_1___Region_Code_Labels_4    0.000152
Policy_Sales_Channel_Label_1___Region_Code_Labels_5    0.000970
Policy_Sales_Channel_Label_2___Region_Code_Labels_2    0.000268
Policy_Sales_Channel_Label_2___Region_Code_Labels_3    0.000232
Policy_Sales_Channel_Label_2___Region_Code_Labels_4    0.000019
Policy_Sales_Channel_Label_2___Region_Code_Labels_5    0.000134
Policy_Sales_Channel_Label_3___Region_Code_Labels_2    0.000226
Policy_Sales_Channel_Label_3___Region_Code_Labels_3    0.000130
Policy_Sales_Channel_Label_3___Region_Code_Labels_4    0.000030
Policy_Sales_Channel_Label_3___Region_Code_Labels_5    0.000288
Policy_Sales_Channel_Label_4___Region_Code_Labels_2    0.000910
Policy_Sales_Channel_Label_4___Region_Code_Labels_3    0.001264
Policy_Sales_Channel_Label_4___Region_Code_Labels_4    0.000272
Policy_Sales_Channel_Label_4___Region_Co

In [101]:
gbm = GradientBoostingClassifier(max_depth=20, learning_rate=0.05, n_estimators=300)
gbm.fit(x_train_undersampled, y_train_undersampled)

GradientBoostingClassifier(learning_rate=0.05, max_depth=20, n_estimators=300)

In [102]:
roc_auc_score(y_test, gbm.predict(x_test)), roc_auc_score(y_train, gbm.predict(x_train))

(0.7120479364539591, 0.7117691026406258)

In [103]:
roc_auc_score(y_train_undersampled, gbm.predict(x_train_undersampled))

0.9985584583654772